# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal `(1)` and abnormal `(0)`. The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
!xournalpp Linear_Classification_Models.pdf 2>/dev/null

In [4]:
!mpv Linear_Classification.mp4 2>&1 1>/dev/null

(Paused) AV: 00:05:38 / 01:29:43 (6%) A-V:  0.000 Dropped: 31K


In [2]:
!mpv Lineare_Klassifikationsmodelle.mp4 2>&1 1>/dev/null

AV: 01:26:37 / 01:27:07 (99%) A-V:  0.0000K


In [2]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function `learn_reg_ERM`$({\color{RawSienna}\mathrm{X}},{\color{Lavender}\mathrm{y}},\mathrm{\color{Tomato}\lambda})$ which for a given $n\times m$ data matrix ${\color{RawSienna}X}$ and binary class label ${{\color{Lavender}y}}$ learns and returns a linear model ${\color{LimeGreen}\theta}={\color{LimeGreen}w}$.
The binary class label has to be transformed so that its range is ${{\color{Lavender}y}}\in\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by ${\color{Tomato}\lambda} > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [ ]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200 # number of iterations
    e  = 0.001 # deviation 
    alpha = 1. # step size

    w = np.random.randn(X.shape[1]) # parameters of the model
    for k in np.arange(max_iter):
        h = np.dot(X,w) # dot product between the data vector and modell vector w
        l,lg = loss(h, y) # loss function output: loss function and gradient of loss function
        print ('loss: {}'.format(np.mean(l)))
        r,rg = reg(w, lbda) # regulazier function output: regulazier function and gradient of regulazier function
        g = np.dot(X.T,lg) + rg # gradient of the sum of the gradients of loss function and regulazier function
        if (k > 0):
            alpha = alpha * ( np.dot(g_old.T, g_old ) )/ ( np.dot((g_old - g).T , g_old ) ) 
        w = w - alpha * g # compute the new parameters w or theta
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

### Exercise 2

Fill in the code for the function `loss`$({\color{CornflowerBlue}h},{\color{Lavender}y}_{\color{VioletRed}i})$ which computes the hinge loss ${\color{CornflowerBlue}l}_\mathrm{h}=\mathrm{\color{CornflowerBlue}l}$ and its gradient ${\color{CornflowerBlue}\nabla l}=\mathrm{\color{CornflowerBlue}g}$. 
This function takes a given vector ${\color{Lavender}y}$ with the true labels ${{\color{Lavender}y}}_{\color{VioletRed}i}\in \left \{-1,1\right \}$ and a vector ${\color{CornflowerBlue}h}={\color{CornflowerBlue}f}({\color{LimeGreen}w},{\color{Lavender}y}_{\color{VioletRed}i})$ with the function values of the linear model as inputs. The function returns a vector $\mathrm{\color{CornflowerBlue}l}$ with the hinge loss 

$${\color{CornflowerBlue}l}_\mathrm{h}=\mathrm{\color{CornflowerBlue}l}={\color{CornflowerBlue}\max}(0, (1 − {{\color{Lavender}y}}_{\color{VioletRed}i} {\color{CornflowerBlue}h}_{\color{VioletRed}i}))$$

and a vector $\mathrm{\color{CornflowerBlue}g}$ with the gradients of the hinge loss with respect to ${\color{CornflowerBlue}h}$. 

Note: The partial derivative of the hinge loss with respect to ${\color{CornflowerBlue}h}$  is 

$$
{\color{CornflowerBlue}\nabla l}=\mathrm{\color{CornflowerBlue}g}= \;
\begin{aligned}
{\color{CornflowerBlue}g}_{\color{VioletRed}i} & = −{\color{Lavender}y} & \mathrm{if} \; {\color{CornflowerBlue}l}_{\color{VioletRed}i} > 0, \\
{\color{CornflowerBlue}g}_{\color{VioletRed}i} & = 0  & \mathrm{else} 
\end{aligned}
$$

In [7]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    l=np.zeros(len(y))
    g=np.zeros(len(y))
    for i in range(len(y)):
        if 1-y[i]*h[i]>0:
            l[i]=1-y[i]*h[i]
            g[i]=-y[i]
        else:
            l[i]=0
            g[i]=0
    return l, g

### Exercise 3

Fill in the code for the function `reg`$({\color{LimeGreen}w}, {\color{Tomato}\lambda})={\color{CornflowerBlue}\Omega}({\color{LimeGreen}w}, {\color{Tomato}\lambda})$ which computes the $\mathcal{L}_2$-regularizer${\color{CornflowerBlue}\Omega}({\color{LimeGreen}w}, {\color{Tomato}\lambda})= \mathrm{\color{CornflowerBlue}r} $ and the gradient ${\color{CornflowerBlue}\nabla \Omega}({\color{LimeGreen}w}, {\color{Tomato}\lambda})= \mathrm{\color{CornflowerBlue}g}$ of the regularizer function at point ${\color{LimeGreen}w}$.

$$ \mathrm{\color{CornflowerBlue}r} = \frac{\color{Tomato}\lambda}{2} {\color{LimeGreen}w}^{\mathrm{T}}{\color{LimeGreen}w}$$

$$\mathrm{\color{CornflowerBlue}g} = {\color{Tomato}\lambda} {\color{LimeGreen}w}$$

In [8]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    ##################
    r = lbda/2*np.dot(w,w)
    g = ldba*w
    return r, g

### Exercise 4

Fill in the code for the function `predict`$({\color{LimeGreen}\mathrm{w}}, {\color{RawSienna}\mathrm{x}})$ which predicts the class label ${\color{Lavender}\mathrm{y}}$ for a data point ${\color{RawSienna}\mathrm{x}}$ or a matrix ${\color{RawSienna}\mathrm{X}}$ of data points (row-wise) for a previously trained linear model ${\color{LimeGreen}\mathrm{w}}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [ ]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    ##################
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [ ]:
##################
#INSERT CODE HERE#
##################

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [ ]:
##################
#INSERT CODE HERE#
##################